<a href="https://colab.research.google.com/github/Accenture-1a-BTT/accenture-1a-team-project/blob/main/FakeNews_Accenture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello World! This is Workspace Test!")

Hello World! This is Workspace Test!


In [2]:
print("This is Test 2 for Version Control")

This is Test 2 for Version Control
